<a href="https://colab.research.google.com/github/Shivani8780/Matrimonial-Profile/blob/main/matching_algo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re

# Load your data (Replace 'your_file.csv' with your actual file path)
df = pd.read_csv('matching.csv')  # Replace with the path to your actual CSV file

# Function to clean height values
def clean_height(value):
    # Convert to string to handle any type of input
    value = str(value)

    # Step 1: Remove all non-numeric characters except the decimal point (.)
    cleaned_value = re.sub(r"[^0-9.]", "", value)

    # Step 2: Ensure only one decimal point exists, if multiple, remove the extras
    if cleaned_value.count(".") > 1:
        cleaned_value = cleaned_value.replace(".", "", cleaned_value.count(".") - 1)

    # Step 3: Convert the cleaned value to float if possible
    try:
        return float(cleaned_value) if cleaned_value else None
    except ValueError:
        return None

# Apply the cleaning function to the 'Height (In Feet)' column
df['Cleaned_Height'] = df['Height (In Feet)'].apply(clean_height)

# Display the cleaned DataFrame
print(df)

# Optionally, save the cleaned data to a new CSV file
df.to_csv('cleaned_data.csv', index=False)  # This saves the cleaned data to 'cleaned_data.csv'


            Name Of Candidate Candidate Gender      Candidate Current City   \
0                      Bhavik              Male                    Vadodara   
1                 Jigar Mehta              Male                   Ahmedabad   
2               Maulik S Jani              Male                   Ahmedabad   
3                  Arpan Vyas              Male  United arab emirates Dubai   
4     Niyati Hitendra Pandya             Female                  Ahmedabad    
...                       ...               ...                         ...   
1760              Deven Joshi              Male                    Vadodara   
1761       Er. Narayan Shukla              Male                    Vadodara   
1762  MEHTA RUSHI BHARATBHAI               Male                  AHMEDABAD    
1763             Sachin Raval              Male             Kassel, Germany   
1764            Neel Upadhyay              Male    Toronto, Ontario, Canada   

     Candidate Current Country  Visa or Residence S

In [ ]:
import pandas as pd
import re

# Load your dataset (replace with the actual path to your CSV file)
df = pd.read_csv('cleaned_data.csv')  # Replace with your CSV file path

# Function to clean and format dates
def clean_date(date_str):
    try:
        # Check if the date contains no separator (e.g., 21101977)
        if isinstance(date_str, str) and len(date_str) == 8 and date_str.isdigit():
            # Convert the date without separator into DD-MM-YYYY format
            return f"{date_str[:2]}-{date_str[2:4]}-{date_str[4:]}"

        # If it's not a non-separated date, try to parse it
        cleaned_date = pd.to_datetime(date_str, errors='coerce', dayfirst=True)  # Set dayfirst=True if dates are in DD/MM/YYYY format
        return cleaned_date.strftime('%d-%m-%Y') if pd.notna(cleaned_date) else None  # Format as DD-MM-YYYY
    except Exception as e:
        return None  # Return None if there's an error in conversion

# Apply the cleaning function to the 'Date Of Birth' column
df['Cleaned_Date_Of_Birth'] = df['Date Of Birth'].apply(clean_date)

# Display the cleaned DataFrame
print(df)

# Optionally, save the cleaned data to a new CSV file
df.to_csv('cleaned_data.csv', index=False)  # Save cleaned data to 'cleaned_data.csv'


<ipython-input-15-027314f9fd46>:16: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  cleaned_date = pd.to_datetime(date_str, errors='coerce', dayfirst=True)  # Set dayfirst=True if dates are in DD/MM/YYYY format


            Name Of Candidate Candidate Gender      Candidate Current City   \
0                      Bhavik              Male                    Vadodara   
1                 Jigar Mehta              Male                   Ahmedabad   
2               Maulik S Jani              Male                   Ahmedabad   
3                  Arpan Vyas              Male  United arab emirates Dubai   
4     Niyati Hitendra Pandya             Female                  Ahmedabad    
...                       ...               ...                         ...   
1760              Deven Joshi              Male                    Vadodara   
1761       Er. Narayan Shukla              Male                    Vadodara   
1762  MEHTA RUSHI BHARATBHAI               Male                  AHMEDABAD    
1763             Sachin Raval              Male             Kassel, Germany   
1764            Neel Upadhyay              Male    Toronto, Ontario, Canada   

     Candidate Current Country  Visa or Residence S

In [ ]:
import pandas as pd
path = 'cleaned_data.csv'
df1 = pd.read_csv(path)
df1["Education Details"].nunique()

1456

In [ ]:
import pandas as pd
import re

# Load your data (replace the filename with your actual file path)
file_path = 'cleaned_data.csv'  # Update this to your actual file path
df = pd.read_csv(file_path)  # Use pd.read_excel() if your file is an Excel file

# Function to clean and standardize education details
def clean_education(education):
    # Remove extra spaces
    education = education.strip() if isinstance(education, str) else ''

    # Convert to uppercase for uniformity
    education = education.upper()

    # Standardize common education terms
    education = re.sub(r'\bB\.A\.\b|\bBA\b', 'BACHELORS OF ARTS', education)
    education = re.sub(r'\bB\.SC\b|\bBSC\b|\bB\.S\.C\b', 'BACHELORS OF SCIENCE', education)
    education = re.sub(r'\bM\.A\.\b|\bMA\b', 'MASTERS OF ARTS', education)
    education = re.sub(r'\bM\.SC\b|\bMSC\b|\bM\.S\.C\b', 'MASTERS OF SCIENCE', education)
    education = re.sub(r'\bMBA\b', 'MASTER OF BUSINESS ADMINISTRATION', education)
    education = re.sub(r'\bPHD\b', 'DOCTORATE', education)

    # Categorize based on keywords
    if 'BACHELOR' in education:
        return 'Bachelors'
    elif 'MASTER' in education or 'MBA' in education:
        return 'Masters'
    elif 'DOCTORATE' in education or 'PHD' in education:
        return 'Doctorate'
    elif 'HIGH SCHOOL' in education:
        return 'High School'
    else:
        return 'Other'  # For any unrecognized values

# Apply the function to clean the Education column
df['Cleaned_Education'] = df['Education Details'].apply(clean_education)

# Save the cleaned data back to the same input file (overwriting the file)
df.to_csv(file_path, index=False)

# Optionally, print the first few rows to check if cleaning worked
print(df.head())


         Name Of Candidate Candidate Gender      Candidate Current City   \
0                   Bhavik              Male                    Vadodara   
1              Jigar Mehta              Male                   Ahmedabad   
2            Maulik S Jani              Male                   Ahmedabad   
3               Arpan Vyas              Male  United arab emirates Dubai   
4  Niyati Hitendra Pandya             Female                  Ahmedabad    

  Candidate Current Country  Visa or Residence Status Of Candidate   \
0                     India                          Indian Citizen   
1                     India                          Indian Citizen   
2                     India                          Indian Citizen   
3                     Dubai                       NRI - Work Permit   
4                     India                          Indian Citizen   

  Date Of Birth  Weight (In KG)  \
0    11.09.1989              80   
1      21101977              75   
2    16-07-

In [ ]:
import pandas as pd
import re

# Load your data (replace the filename with your actual file path)
file_path = '/content/cleaned_data (2).csv'  # Update this to your actual file path
df = pd.read_csv(file_path)  # Use pd.read_excel() if your file is an Excel file

# Function to clean and standardize education details
def clean_education(education):
    # Ensure the input is a string and strip extra spaces
    education = str(education).strip().upper()

    # Check for specific patterns
    if education.startswith('B'):  # Starts with 'B'
        return 'Bachelors'
    elif education.startswith('M'):  # Starts with 'M'
        return 'Masters'
    elif education.startswith('10') or education.startswith('10TH'):  # Starts with '10' or '10th'
        return 'Secondary School'
    elif education.startswith('12') or education.startswith('12TH'):  # Starts with '12' or '12th'
        return 'High School'
    elif 'DIPLOMA' or "Diploma" in education:  # Contains 'Diploma'
        return 'Diploma'
    elif 'PHD' in education or 'DOCTORATE' in education:  # Contains 'PhD' or 'Doctorate'
        return 'Doctorate'
    else:
        return 'Other'  # Default category for unrecognized values

# Apply the function to clean the Education column
df['Cleaned_Education'] = df['Education Details'].apply(clean_education)

# Save the cleaned data back to the same input file (overwriting the file)
df.to_csv(file_path, index=False)

# Optionally, print the first few rows to verify the changes
print(df.head())


         Name Of Candidate Candidate Gender      Candidate Current City   \
0                   Bhavik              Male                    Vadodara   
1              Jigar Mehta              Male                   Ahmedabad   
2            Maulik S Jani              Male                   Ahmedabad   
3               Arpan Vyas              Male  United arab emirates Dubai   
4  Niyati Hitendra Pandya             Female                  Ahmedabad    

  Candidate Current Country  Visa or Residence Status Of Candidate   \
0                     India                          Indian Citizen   
1                     India                          Indian Citizen   
2                     India                          Indian Citizen   
3                     Dubai                       NRI - Work Permit   
4                     India                          Indian Citizen   

  Date Of Birth  Weight (In KG)  \
0    11.09.1989              80   
1      21101977              75   
2    16-07-

In [ ]:
# Load the data
file_path = 'cleaning_Data.csv'  # Replace this with your actual file path if needed
df1 = pd.read_csv(file_path)  # Adjust if your file is a different format like Excel
df.head()

,Name Of Candidate,Candidate Gender,Candidate Current City,Candidate Current Country,Visa or Residence Status Of Candidate,Date Of Birth,Weight (In KG),Education Details,Details on Job / Business / Occupation,Income,...,Choice Of Education : (Who should contact you),"Choice Of Location : (Who should contact you) ; Eg. Only Amdavad , Vadodara , Mumbai , NRI , Gujarat Side , Saurashtra Kachchh side , etc)",Choice Of Age Gap/ Difference in Year,Any Other Choice,Height,Cleaned_Height,Final_Heiht,Cleaned_Date_Of_Birth,Weight(in kg),Cleaned_Education
0,Bhavik,Male,Vadodara,India,Indian Citizen,11.09.1989,80,Bca,Job,NaN,...,Any,Any,30.34,No,5.0,5.06,5.06,11-09-1989,80.0,Bachelors
1,Jigar Mehta,Male,Ahmedabad,India,Indian Citizen,21101977,75,BA With Economic,Job,16 lac per annum,...,Graduate,All gujarat,42 to 45,No,NaN,5.07,5.07,21-10-1977,75.0,Bachelors
2,Maulik S Jani,Male,Ahmedabad,India,Indian Citizen,16-07-1991,70,Bsc. Chemistry,Clerck At RTO Department,NaN,...,Graduate any,Gujarat any,26-32,Less Practical More Emotional,NaN,5.80,5.8,16-07-1991,70.0,Bachelors
3,Arpan Vyas,Male,United arab emirates Dubai,Dubai,NRI - Work Permit,17-12-1991,80,10th Only because of some issue couldnr comple...,SALES MANAGER IN DIAMOND JEWELLERY SHOWROOM DUBAI,1Lac Per Month,...,Everyone who dont have problem with my educati...,Any,2 -3 yeaes younger or same age preferred,Notrhing,NaN,5.11,5.11,17-12-1991,80.0,Secondary School
4,Niyati Hitendra Pandya,Female,Ahmedabad,India,Indian Citizen,30-Mar-97,71,MBBS,Jivraj Mehta Hospital,50k,...,"Md,MS,Higher post candidate, classone Officer,...",All,27 to 31,Handdome,NaN,5.40,5.4,30-03-1997,71.0,Masters


In [ ]:
# Print column names to verify
print(df1.columns)


Index([' Name Of Candidate', 'Candidate Gender ', 'Candidate Current City ',
       'Candidate Current Country', ' Visa or Residence Status Of Candidate ',
       'Date Of Birth', ' Weight (In KG)', 'Education Details',
       ' Details on Job / Business / Occupation', 'Income ', 'Marriage Status',
       'Gotra',
       'Candidate has Any Mental / Physical Disability or Minor Problem  ?',
       'Choice Of Education : (Who should contact you)',
       'Choice Of Location :  (Who should contact you) ; Eg. Only Amdavad , Vadodara , Mumbai , NRI , Gujarat Side , Saurashtra Kachchh side , etc)',
       'Choice Of Age Gap/ Difference in Year ', 'Any Other Choice ', 'Height',
       'Cleaned_Height', 'Final_Heiht', 'Cleaned_Date_Of_Birth',
       'Weight(in kg)', 'Cleaned_Education'],
      dtype='object')


In [ ]:
import pandas as pd
import re

# Sample data (load your own data instead of this)
df1 = pd.read_csv('cleaning_Data.csv')

# Function to clean and standardize income details
def clean_income(income):
    # Check for empty or NaN values
    if pd.isna(income) or str(income).strip() == '':
        return None  # Return None for empty or missing data

    # Convert income to string for easier manipulation
    income = str(income).strip().upper()

    # Handle "Does not want to specify" or similar entries
    if "DOES NOT WANT TO SPECIFY" in income:
        return None  # or you can return 'Unknown'

    # Handle "Per Month" or "Per Hour" to convert to annual
    if 'PER MONTH' in income or 'PER HOUR' in income:
        # Extract numerical part and convert to annual if per month
        match = re.search(r'(\d+[\.,]?\d*)', income)
        if match:
            amount = match.group(1).replace(",", "")
            if "PER MONTH" in income:
                return float(amount) * 12  # Convert monthly to annual
            elif "PER HOUR" in income:
                return float(amount) * 12 * 160  # Assume 160 hours per month for per-hour

    # Handle "Lac" and "Lakh" as Lakhs
    if 'LAC' in income or 'LAKH' in income:
        match = re.search(r'(\d+[\.,]?\d*)', income)
        if match:
            amount = match.group(1).replace(",", "")
            return float(amount) * 100000  # Convert lakh/lac to INR (in lakhs)

    # Handle "CAD" and "USD" for foreign currencies
    if 'CAD' in income or 'USD' in income:
        match = re.search(r'(\d+[\.,]?\d*)', income)
        if match:
            amount = match.group(1).replace(",", "")
            return float(amount) * 80  # Assuming exchange rate of 1 CAD/USD = 80 INR

    # Handle straight numeric values
    match = re.search(r'(\d+[\.,]?\d*)', income)
    if match:
        amount = match.group(1).replace(",", "")
        return float(amount)

    # If none of the above, return None or 'Other'
    return None

# Apply the function to clean the income column
df1['Cleaned_Income'] = df1['Income '].apply(clean_income)

# Replace any NaN in 'Cleaned_Income' with None or a default value (e.g., 0)
df1['Cleaned_Income'] = df1['Cleaned_Income'].fillna('Unknown')

# Save the cleaned data back to the same file (overwrite)
df1.to_csv('cleaning_Data.csv', index=False)

# Optionally, print the first few rows to verify the changes
print(df1.head())


         Name Of Candidate Candidate Gender      Candidate Current City   \
0                   Bhavik              Male                    Vadodara   
1              Jigar Mehta              Male                   Ahmedabad   
2            Maulik S Jani              Male                   Ahmedabad   
3               Arpan Vyas              Male  United arab emirates Dubai   
4  Niyati Hitendra Pandya             Female                  Ahmedabad    

  Candidate Current Country  Visa or Residence Status Of Candidate   \
0                     India                          Indian Citizen   
1                     India                          Indian Citizen   
2                     India                          Indian Citizen   
3                     Dubai                       NRI - Work Permit   
4                     India                          Indian Citizen   

  Date Of Birth  Weight (In KG)  \
0    11.09.1989              80   
1      21101977              75   
2    16-07-

In [ ]:
# Print column names to verify
print(df1['Cleaned_Income'].head())

0      Unknown
1    1600000.0
2      Unknown
3         12.0
4         50.0
Name: Cleaned_Income, dtype: object


In [ ]:
import pandas as pd
df1 = pd.read_csv('cleaning_Data.csv')
df1.head()

,Name Of Candidate,Candidate Gender,Candidate Current City,Candidate Current Country,Visa or Residence Status Of Candidate,Date Of Birth,Weight (In KG),Education Details,Details on Job / Business / Occupation,Income,...,"Choice Of Location : (Who should contact you) ; Eg. Only Amdavad , Vadodara , Mumbai , NRI , Gujarat Side , Saurashtra Kachchh side , etc)",Choice Of Age Gap/ Difference in Year,Any Other Choice,Height,Cleaned_Height,Final_Heiht,Cleaned_Date_Of_Birth,Weight(in kg),Cleaned_Education,Cleaned_Income
0,Bhavik,Male,Vadodara,India,Indian Citizen,11.09.1989,80,Bca,Job,NaN,...,Any,30.34,No,5.0,5.06,5.06,11-09-1989,80.0,Bachelors,Unknown
1,Jigar Mehta,Male,Ahmedabad,India,Indian Citizen,21101977,75,BA With Economic,Job,16 lac per annum,...,All gujarat,42 to 45,No,NaN,5.07,5.07,21-10-1977,75.0,Bachelors,1600000.0
2,Maulik S Jani,Male,Ahmedabad,India,Indian Citizen,16-07-1991,70,Bsc. Chemistry,Clerck At RTO Department,NaN,...,Gujarat any,26-32,Less Practical More Emotional,NaN,5.80,5.8,16-07-1991,70.0,Bachelors,Unknown
3,Arpan Vyas,Male,United arab emirates Dubai,Dubai,NRI - Work Permit,17-12-1991,80,10th Only because of some issue couldnr comple...,SALES MANAGER IN DIAMOND JEWELLERY SHOWROOM DUBAI,1Lac Per Month,...,Any,2 -3 yeaes younger or same age preferred,Notrhing,NaN,5.11,5.11,17-12-1991,80.0,Secondary School,12.0
4,Niyati Hitendra Pandya,Female,Ahmedabad,India,Indian Citizen,30-Mar-97,71,MBBS,Jivraj Mehta Hospital,50k,...,All,27 to 31,Handdome,NaN,5.40,5.4,30-03-1997,71.0,Masters,50.0


In [ ]:
import pandas as pd
import re

# Load the CSV file
file_path = "cleaning_Data"  # Replace with your CSV file path
df = pd.read_csv(file_path)

# Function to extract salary values
def extract_salaries(text):
    salary_pattern = r"(\d+(?:,\d{3})*(?:\.\d+)?)(?:\s*(?:lac|lakh|lpa|year|month|cad|inr|usd|per|annum|annual|per\s*month))?"
    matches = re.findall(salary_pattern, text, re.IGNORECASE)
    salary_values = []

    for match in matches:
        value = match[0]
        unit = match[1].lower() if len(match) > 1 else ''

        # Convert numeric string to float
        value = value.replace(",", "")
        value = float(value)

        # Normalize values to annual income in INR
        if "month" in unit or "per month" in unit:
            value = value * 12  # Monthly to annual conversion
        elif "lac" in unit or "lakh" in unit:
            value = value * 100000  # Convert lakh to INR
        elif "lpa" in unit:
            value = value * 100000  # Convert LPA to INR annually

        salary_values.append(value)

    return salary_values

# Column that contains the salary data in your CSV
salary_column = 'Income '  # Replace with the actual column name from your CSV

# Apply salary extraction to the column
df['Extracted_Salary'] = df[salary_column].apply(lambda x: extract_salaries(str(x)))

# Print the resulting dataframe
print(df[['Salary', 'Extracted_Salary']])

# Save the updated dataframe to a new CSV file (optional)
df.to_csv('updated_file.csv', index=False)
